In [1]:
import pandas as pd
import numpy as np
from kaggle_handler import handler

# Download and load Dataset

    This Dataset is a toy dataset and only being used for learning purpose

In [2]:
Assets = handler("ashaychoudhary/gen-z-dating-apps-india")

Directory 'Assets' already exists.
Datasets already exist in Assets folder
['tested.csv', 'GenZ_DatingApp_Data.csv']
 Change Add_more parameter to download more datasets


In [3]:
Assets

['tested.csv', 'GenZ_DatingApp_Data.csv']

In [4]:
df = pd.read_csv('Assets/GenZ_DatingApp_Data.csv',usecols=['Gender', 'Location', 'Education', 'Occupation'])

# Dataset Over View
- Odinal Data Columns [Education]
- Nominal Data Columns [Gender, Location]

In [5]:
df.head(5)

,Gender,Location,Education,Occupation
0,Non-binary,Bangalore,Undergraduate,Freelancer
1,Female,Delhi,Undergraduate,Part-time Job
2,Non-binary,Kolkata,Undergraduate,Intern
3,Non-binary,Delhi,Graduate,Full-time Job
4,Male,Delhi,Graduate,Intern


In [6]:
df.shape

(500, 4)

# Spliting Dataset

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Gender','Location','Education']],df['Occupation'],test_size=.2)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((400, 3), (400,), (100, 3), (100,))

# Ordinal Encoding

In [8]:
X_train.head(5)

,Gender,Location,Education
304,Female,Mumbai,Graduate
272,Non-binary,Kolkata,Postgraduate
185,Non-binary,Kolkata,Graduate
53,Male,Delhi,Postgraduate
221,Female,Delhi,Undergraduate


In [9]:
X_train['Education'].unique()

array(['Graduate', 'Postgraduate', 'Undergraduate'], dtype=object)

In [10]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(categories=[['Undergraduate', 'Graduate', 'Postgraduate']], dtype=np.int32)
oe.fit(X_train[['Education']])

X_train_oe = oe.transform(X_train[['Education']])
X_test_oe = oe.transform(X_test[['Education']])

In [11]:
oe.categories

[['Undergraduate', 'Graduate', 'Postgraduate']]

In [12]:
X_train['Education'] = np.reshape(X_train_oe, (X_train_oe.shape[0],))
X_test['Education'] = np.reshape(X_test_oe, (X_test_oe.shape[0],))

In [13]:
X_train.head(5)

,Gender,Location,Education
304,Female,Mumbai,1
272,Non-binary,Kolkata,2
185,Non-binary,Kolkata,1
53,Male,Delhi,2
221,Female,Delhi,0


# Label Encoding

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

In [15]:
le.classes_

array(['Freelancer', 'Full-time Job', 'Intern', 'Part-time Job',
       'Student'], dtype=object)

In [16]:
pd.DataFrame(y_train, columns=['Occupation']).head(5)

,Occupation
304,Freelancer
272,Full-time Job
185,Full-time Job
53,Student
221,Freelancer


In [17]:
pd.DataFrame(y_train_le, columns=['Occupation']).head(5)

,Occupation
0,0
1,1
2,1
3,4
4,0


# OneHot Encoding

In [18]:
X_train.head(5)

,Gender,Location,Education
304,Female,Mumbai,1
272,Non-binary,Kolkata,2
185,Non-binary,Kolkata,1
53,Male,Delhi,2
221,Female,Delhi,0


In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first',sparse_output=False,dtype=np.int32)

X_train_ohe = ohe.fit_transform(X_train[['Gender','Location']])
X_test_ohe = ohe.transform(X_test[['Gender','Location']])

In [20]:
ohe.categories_

[array(['Female', 'Male', 'Non-binary'], dtype=object),
 array(['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai',
        'Pune'], dtype=object)]

In [21]:
X_train_ohe.shape

(400, 8)

In [22]:
new_df = pd.DataFrame(X_train_ohe, columns=np.concat((np.delete(ohe.categories_[0],0),
                                             np.delete(ohe.categories_[1],0)),
                                            axis=0)
            )

In [23]:
new_df.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_train = pd.concat([new_df,X_train['Education']], axis=1)
X_train.head(5)

,Male,Non-binary,Chennai,Delhi,Hyderabad,Kolkata,Mumbai,Pune,Education
0,0,0,0,0,0,0,1,0,1
1,0,1,0,0,0,1,0,0,2
2,0,1,0,0,0,1,0,0,1
3,1,0,0,1,0,0,0,0,2
4,0,0,0,1,0,0,0,0,0


# Pandas In-Build Function

    We don't use it in mashine learning becasue it dos't remamber tha changes it make

In [24]:
X_test.head(5)

,Gender,Location,Education
377,Female,Kolkata,0
365,Female,Hyderabad,1
333,Female,Delhi,2
275,Male,Mumbai,1
278,Male,Mumbai,0


In [25]:
pd.get_dummies(X_test, columns=['Gender','Location'],drop_first=True,dtype=np.int32)

,Education,Gender_Male,Gender_Non-binary,Location_Chennai,Location_Delhi,Location_Hyderabad,Location_Kolkata,Location_Mumbai,Location_Pune
377,0,0,0,0,0,0,1,0,0
365,1,0,0,0,0,1,0,0,0
333,2,0,0,0,1,0,0,0,0
275,1,1,0,0,0,0,0,1,0
278,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
251,2,1,0,0,0,0,0,0,0
96,0,0,0,0,1,0,0,0,0
297,0,0,0,0,0,0,0,0,1
461,1,0,0,0,0,0,0,0,1
